In [ ]:
%run feature_extraction.ipynb

In [ ]:
# imports
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import tqdm
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import torch

In [ ]:
#imports
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# reading train data
train_1=pd.read_pickle('output/train_3_original.pkl')

In [ ]:
# reading dev data
dev_1=pd.read_pickle('output/dev_1_original.pkl')

In [ ]:
#check reading is fine
train_1.head(10)


In [ ]:
# change  category to be int
def edit_categories(x):
    if x == 'info_news':
        return 0
    elif x == 'celebrity':
        return 1
    elif x == 'plan':
        return 2
    elif x == 'requests':
        return 3
    elif x == 'rumors':
        return 4
    elif x == 'advice':
        return 5
    elif x == 'restrictions':
        return 6
    elif x == 'personal':
        return 7
    elif x == 'unrelated':
        return 8
    elif x == 'others':
        return 9
    else:
        return -1

In [ ]:
# passing values for train
sentences_train,stance_train,category_train =train_1.loc[:,"tokens"],train_1.loc[:,"stance"],train_1.loc[:,"category"].apply(edit_categories)

In [ ]:
#passing values for test
sentences_dev,stance_dev,category_dev =dev_1.loc[:,"tokens"],dev_1.loc[:,"stance"],dev_1.loc[:,"category"].apply(edit_categories)

In [ ]:
#check passing is fine
print(len(sentences_train),len(category_train),len(stance_train))

In [ ]:
#check passing is fine
print(len(sentences_dev),len(category_dev),len(stance_dev))

In [ ]:
new_sentences_train=[]
new_sentences_dev=[]
# merge tokens to be sentences to can be used in some features
for sentence in sentences_train:
    new_sentences_train.append(' '.join(sentence))
for sentence in sentences_dev:
    new_sentences_dev.append(' '.join(sentence))

In [ ]:
# for tf-idf , you should send sentences
def preprocessing_tf_ifd(sentences):
    new_sentences=[]
    for sentence in sentences:
        new_sentences.append(' '.join(sentence))
    return new_sentences


In [ ]:
#get  new_sentences_train
new_sentences_train=preprocessing_tf_ifd(sentences_train)

In [ ]:
#get tf -idf for training dta 

dict_res_train,tfIdf,tf_idf_train,tfIdfVectorizer=get_TF_IDF(new_sentences_train)

In [ ]:
# get unique words in training which is used in tf -df
unique_words_train= tfIdfVectorizer.get_feature_names()

In [ ]:
print(len(unique_words_train))

In [ ]:
# for test or dev data , assume some words are new , so change them with word in train
#choosing word is important , currently i random it with word has good probabitiy 
def change_test_set(sentences_dev, unique_words_train, changed_word ='قوة'):
    new_sentences=[]
    for sentence in sentences_dev:
        new_sentence=[]
        for word in sentence:
            if ( word not in unique_words_train):
                word = changed_word
            new_sentence.append(word)
        new_sentences.append(new_sentence)
    return new_sentences

In [ ]:
#change sentences of dev for tf -idf 
# don't use them for n gram , in ngram i am doing smoothing
sentences_dev_removed=change_test_set(sentences_dev,unique_words_train)

In [ ]:
# get train data as sentences
for sentence in sentences_train:
    new_sentences_train.append(' '.join(sentence))

In [ ]:

new_sentences_dev_removed=[]
#using new sentences of dev after removing new words by existing word
# then convert tokens to sentences
for sentence in sentences_dev_removed:
    new_sentences_dev_removed.append(' '.join(sentence))
tf_idf_dev=[]
# using the same transformer in testing data
tfIdf=tfIdfVectorizer.fit_transform(new_sentences_dev_removed)
# here i get tf -idf for test
for i in range(0,len(new_sentences_dev_removed)):
        df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
        dict_res= preprocessing_tf_idf(i,tfIdfVectorizer,tfIdf)
        tf_idf_currrent =[]
        #here making sure the tf -idf for test with the same size of train
        for word in unique_words_train:
            if word in dict_res:
                tf_idf_currrent.append(dict_res[word])
            else:
                tf_idf_currrent.append(0.0)
        tf_idf_dev.append(tf_idf_currrent)


In [ ]:
# converting all to lists
for i  in range(0,len(tf_idf_train)):
    tf_idf_train[i]= list(tf_idf_train[i])
for i in range(0,len(tf_idf_dev)):
    tf_idf_dev[i]= list(tf_idf_dev[i])

In [ ]:
#check they are the same
print(len(tf_idf_dev[0]), len(tf_idf_train[0]))

In [ ]:
#get n gram for data
prob_dev = convert_ngrams_probablities( 2,new_sentences_dev)
prob_train = convert_ngrams_probablities( 2,new_sentences_train)

In [ ]:
#currently i am using one feature change it when u want
feature_train = tf_idf_train
feature_dev =tf_idf_dev

In [ ]:
#this doesn't work
# feature_train=[[]]*3000
# for i in range(0,3000):
#     feature_train[i].append(tf_idf_train[i])
#     feature_train[i].append(prob_train[i])

In [28]:
#models imports
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from  matplotlib import pyplot 

In [29]:
values =np.linspace(1, 1000, num=1000)

In [30]:
# Boost 100 weak decision trees

n_estimators = [50, 100, 150, 200, 250, 300, 350]
train_scores=[]
test_scores=[]
for val in n_estimators:
    model = RandomForestClassifier(n_estimators=val)
    model = model.fit(feature_train, stance_train)
    predictions = model.predict(feature_dev)
    acc = {metrics.accuracy_score(stance_dev, predictions) * 100}
    print(f" Model Accuracy is {acc}")
    print(len(predictions[stance_dev==0]))
    print(len(predictions[stance_dev==1]))
    print(len(predictions[stance_dev==2]))
    # print(metrics.classification_report(stance_dev,predictions))
    train_score = accuracy_score(stance_train,model.predict(feature_train))
    train_scores.append(train_score)
    test_score = accuracy_score(stance_dev,model.predict(feature_dev))
    test_scores.append(test_score)

 Model Accuracy is {69.5}
70
126
804
 Model Accuracy is {70.0}
70
126
804
 Model Accuracy is {70.5}
70
126
804
 Model Accuracy is {70.1}
70
126
804
 Model Accuracy is {70.7}
70
126
804
 Model Accuracy is {71.0}
70
126
804


In [ ]:
# plot of train and test scores vs tree depth
pyplot.plot(n_estimators, train_scores, '-o', label='Train')
pyplot.plot(n_estimators, test_scores, '-o', label='Test')
pyplot.legend()
pyplot.show()

In [ ]:
cm = confusion_matrix(stance_dev, predictions)


In [ ]:
print(cm)

In [ ]:
# Boost 100 weak decision trees
model = RandomForestClassifier(n_estimators=100)
#GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)

model = model.fit(feature_train, category_train)

In [ ]:
predictions = model.predict(feature_dev)
acc = {metrics.accuracy_score(category_dev, predictions) * 100}
train_score = accuracy_score(category_train,model.predict(feature_train))
print("train accuarcy", train_score)
print(f" Model Accuracy is {acc}")

In [ ]:
print(metrics.classification_report(category_dev,predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, confusion_matrix


In [ ]:
import time
from sklearn import svm,metrics


classifier_linear = svm.SVC(kernel='linear')

classifier_linear.fit(feature_train, stance_train)

predictions  = classifier_linear.predict(feature_dev)
train_score = accuracy_score(stance_train,model.predict(feature_train))
print("train accuarcy", train_score)
print(metrics.accuracy_score(stance_dev, predictions) * 100)



In [ ]:
print(metrics.classification_report(stance_dev,predictions))

In [ ]:

classifier_linear = svm.SVC(kernel='linear')

classifier_linear.fit(feature_train, category_train)
predictions  = classifier_linear.predict(feature_dev)
train_score = accuracy_score(category_train,model.predict(feature_train))
print("train accuarcy", train_score)
print(metrics.accuracy_score(category_dev, predictions) * 100)


In [ ]:
print(metrics.classification_report(category_dev,predictions))

In [ ]:
# Create model(naive bayes) and training. 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(feature_train, stance_train)

In [ ]:
predicted = clf.predict(feature_dev)
# Model Accuracy, how often is the classifier correct?
train_score = accuracy_score(stance_train,model.predict(feature_train))
print("train accuarcy", train_score)
print("Accuracy:",metrics.accuracy_score(stance_dev, predicted)*100)
print(len(predicted[stance_dev==0]))
print(len(predicted[stance_dev==1]))
print(len(predicted[stance_dev==2]))

In [ ]:
# Create model(naive bayes) and training. 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(feature_train, category_train)

In [ ]:
predicted = clf.predict(feature_dev)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(category_dev, predicted)*100)
train_score = accuracy_score(category_train,model.predict(feature_train))
print("train accuarcy", train_score)

In [ ]:
kf =KFold(n_splits=10, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(features, stance):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

In [ ]:
model = RandomForestClassifier(n_estimators=300)

In [ ]:
n_estimators = [50, 100, 150, 200, 250, 300, 350]

for val in n_estimators:
    score = cross_val_score(RandomForestClassifier(n_estimators= val, random_state= 42) ,features, stance, cv= kf, scoring="accuracy")
    print(f'Average score({val}): {"{:.3f}".format(score.mean())}')

In [ ]:
n_estimators = [50, 100, 150, 200, 250, 300, 350]

for val in n_estimators:
    score = cross_val_score(RandomForestClassifier(n_estimators= val, random_state= 42) ,features, category, cv= kf, scoring="accuracy")
    print(f'Average score({val}): {"{:.3f}".format(score.mean())}')


In [ ]:
classifier_linear = svm.SVC(kernel='linear')
cross_val_score(classifier_linear,features, category, cv= kf,scoring="accuracy")
print(f'Average score: {"{:.3f}".format(score.mean())}')